In [43]:
#setup phase
!pip install wikipedia
import pandas as pd 
import wikipedia as wp
from bs4 import BeautifulSoup
# building the code to scrape the following Wikipedia page, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe
# enter the h1 element in wp.page(h1)
html = wp.page("List of postal codes of Canada: M").html().encode("UTF-8")

# determining the index of the table
df = pd.read_html(html, header = 0)[0]

# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
df = df[df.Borough != 'Not assigned']
# More than one neighborhood can exist in one postal code area. The same ones will be combined into one row with the neighborhoods separated with a comma.
df = df.groupby(['Postal Code', 'Borough'])['Neighborhood'].apply(list).apply(lambda x:', '.join(x)).to_frame().reset_index()
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 
for index, row in df.iterrows():
    if row['Neighborhood'] == 'Not assigned':
        row['Neighborhood'] = row['Borough']
df.head(12)

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [44]:
df.shape

(103, 3)

# Another setup


In [47]:
import requests
import io
!pip install shapely
!pip install geopandas
from shapely.geometry import Point
import geopandas as gpd
import matplotlib.pyplot as plt

     |████████████████████████████████| 931kB 7.6MB/s eta 0:00:01
     |████████████████████████████████| 10.9MB 23.3MB/s eta 0:00:01
     |████████████████████████████████| 14.7MB 23.1MB/s eta 0:00:01


# In order to utilize the Foursquare location data, getting the latitude and the longitude coordinates of each neighborhood:

In [67]:
url="http://cocl.us/Geospatial_data"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))
c.head()



,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


# Renaming the first column to allow merging dataframes on Postal Code, Reordering the columns and showing the dataframe

In [85]:
c.columns = ['Postal Code', 'Latitude', 'Longitude']
df = pd.merge(c, df, on='Postal Code')

# reordering column names and show the dataframe
df = df[['Postal Code', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']]

df.head()


,Postal Code,Borough,Neighborhood,Latitude,Longitude,Coordinates
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,POINT (-79.19435 43.80669)
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,POINT (-79.16050 43.78454)
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,POINT (-79.18871 43.76357)
3,M1G,Scarborough,Woburn,43.770992,-79.216917,POINT (-79.21692 43.77099)
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,POINT (-79.23948 43.77314)
